In [1]:
using Pkg
Pkg.activate("..")
using nFit
using LinearAlgebra, BenchmarkTools, Profile
using ForwardDiff
import LsqFit

  Activating project at `~/development/Projects/nFit.jl`


In [9]:
q(x, p) = p[1]*sin(x) + p[2]*x + p[3]*exp(-x/p[4])

#q2(x, p) = @. p[1] * exp(-(x-p[2])^2/p[3]) + p[4]*exp(-x/p[5]) + p[6]
t=0.0:0.02:10.0
data = [q(x, [5.0, 0.2, 5, 2]) for x in t]

data += (rand(length(data)).- 0.5);

p0 = [6.0, 6, 50.0, 5, 1, 1.0]
lm = Fitter(q, [1.0, 1.0, 1, 1], collect(t), data, MaxIter=100)
update_jacobian!(lm)
lm.J


501×4 Matrix{Float64}:
  0.0         0.0   1.0         0.0
  0.0199987   0.02  0.980199    0.019604
  0.0399893   0.04  0.960789    0.0384316
  0.059964    0.06  0.941765    0.0565059
  0.0799147   0.08  0.923116    0.0738494
  0.0998334   0.1   0.904837    0.0904838
  0.119712    0.12  0.88692     0.106431
  0.139543    0.14  0.869358    0.12171
  0.159318    0.16  0.852144    0.136343
  0.17903     0.18  0.83527     0.150349
  ⋮                             
 -0.403393    9.84  5.32773e-5  0.000524253
 -0.421612    9.86  5.22223e-5  0.000514916
 -0.439662    9.88  5.11883e-5  0.000505744
 -0.457536    9.9   5.01747e-5  0.000496733
 -0.475227    9.92  4.91812e-5  0.000487881
 -0.492728    9.94  4.82073e-5  0.000479184
 -0.510032    9.96  4.72527e-5  0.000470641
 -0.527132    9.98  4.63171e-5  0.000462248
 -0.544021   10.0   4.53999e-5  0.000454003

In [10]:
fit(lm, 1)

([5.045152704132915, 0.1975428142117303, 4.929210597347636, 2.0456462632784183], 6)

In [4]:
lm.J

501×6 Matrix{Float64}:
 0.0  0.0  0.0  1.0       0.0          1.0
 0.0  0.0  0.0  0.999926  0.000419087  1.0
 0.0  0.0  0.0  0.999852  0.000838113  1.0
 0.0  0.0  0.0  0.999778  0.00125708   1.0
 0.0  0.0  0.0  0.999704  0.00167598   1.0
 0.0  0.0  0.0  0.99963   0.00209482   1.0
 0.0  0.0  0.0  0.999556  0.00251359   1.0
 0.0  0.0  0.0  0.999482  0.00293231   1.0
 0.0  0.0  0.0  0.999407  0.00335096   1.0
 0.0  0.0  0.0  0.999333  0.00376955   1.0
 ⋮                                     ⋮
 0.0  0.0  0.0  0.964204  0.198825     1.0
 0.0  0.0  0.0  0.964133  0.199214     1.0
 0.0  0.0  0.0  0.964062  0.199604     1.0
 0.0  0.0  0.0  0.96399   0.199993     1.0
 0.0  0.0  0.0  0.963919  0.200382     1.0
 0.0  0.0  0.0  0.963847  0.200771     1.0
 0.0  0.0  0.0  0.963776  0.20116      1.0
 0.0  0.0  0.0  0.963704  0.201549     1.0
 0.0  0.0  0.0  0.963633  0.201938     1.0

In [ ]:

qs(xs, p) = [lm.f(x, p) for x in xs]
ForwardDiff.jacobian(p ->qs(lm.xarr, p), lm.p)

In [ ]:
# function update_jacobian!(lm::Fitter)
#     qs(xs, p) = [lm.f(x, p) for x in xs]
#     lm.J = ForwardDiff.jacobian(p ->qs(lm.xarr, p), lm.p)

# end
# function update_jacobian2!(lm::Fitter)
#     for j ∈ eachindex(lm.p), i ∈ eachindex(lm.xarr)
#         p1, p2 = lm.p[:], lm.p[:]
#         p1[j] += lm.δ
#         p2[j] -= lm.δ
#         @inbounds lm.J[i, j] = (lm.f(lm.xarr[i], p1) - lm.f(lm.xarr[i], p2))/(2*lm.δ)
        
#     end
# end

@btime begin
p = [6.0, 6, 50.0, 5, 1]
qs(xs, p) = [p[1] * exp(-(x-p[2])^2/p[3]) + p[4]*exp(-x/p[5]) for x in xs]
ForwardDiff.jacobian(p ->qs(t, p), p)
end



In [ ]:
@btime update_jacobian!(lm);
lm.J

In [11]:
curve_fit(q, collect(t), data, [1.0, 1.0, 1, 1])

([5.045152704132915, 0.1975428142117303, 4.929210597347636, 2.0456462632784183], 6)

In [ ]:
@btime LsqFit.curve_fit(q2, collect(t), data,[1.0, 2, 20.0, 4, 3])